In [27]:
import pandas as pd
from llama_index.core import Document, VectorStoreIndex
import openai
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Example DataFrame with restaurant data


In [28]:
# API Key
API_KEY = "6ca95wYJMQGrfd-COZR71UxqB7BhxuCSOaurylwvdKjcljEU9hxosKUCx0-0_TTvFNxqmLscQnFSDECaTUXP1YS_bvo8bbU_kQTkzb-_QK4RyjX9DLHAc6fPD8cKZ3Yx"

# API Endpoint and Headers
url_business = "https://api.yelp.com/v3/businesses
/search"
url_reviews = "https://api.yelp.com/v3/businesses/{}/reviews"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

# Search Parameters for Yelp API
params = {
    "term": "restaurants",
    "location": "San Francisco",
    "limit": 50,  # Number of businesses to retrieve
    "sort_by": "best_match"
}

response = requests.get(url_business, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Extract the list of businesses
    businesses = data.get("businesses", [])

    # Create an empty list to store business data with reviews
    business_data = []

    # Loop through each business and get the reviews
    for biz in businesses:
        # Extract business details
        business_info = {
            "Name": biz["name"],
            "ID": biz["id"],
            "Location": ", ".join(biz["location"]["display_address"]),
            "Rating": biz["rating"],
            "Review Count": biz["review_count"],
            "Phone": biz["phone"],
            "Price": biz.get("price", None),
            "Categories": ", ".join([cat["title"] for cat in biz["categories"]])
        }

        # Send request to get reviews for the business
        review_response = requests.get(url_reviews.format(biz["id"]), headers=headers)

        # Check if the review request was successful
        if review_response.status_code == 200:
            review_data = review_response.json().get("reviews", [])

            # Add reviews to business_info
            business_info["Reviews"] = "; ".join([rev["text"] for rev in review_data])

        # Append the business information (with reviews) to the list
        business_data.append(business_info)

        # Add a delay to avoid hitting the API rate limit
        time.sleep(0.1)

    # Create a DataFrame from the combined business and review data
    df = pd.DataFrame(business_data)
    df.drop(["ID", "Phone"], axis = 1, inplace = True)
    df.dropna(axis=0, inplace=True)
    df['text'] = "Name: " + df['Name'] + "\nLocation: " + df['Location'] + "\nRating: " + df['Rating'].astype(str) + "\n# of reviews: " + df['Review Count'].astype(str) + "\nPrice: " + df['Price'] + "\nCategories: " + df['Categories'] + "\nReviews: " + df['Reviews']
    # Display the DataFrame with reviews
else:
    print(f"Failed to retrieve businesses: {response.status_code}")


df.head(10)
# df.to_csv('my_dataframe.csv', index=False)

# from google.colab import files
# files.download('my_dataframe.csv')


Failed to retrieve businesses: 429


,Name,Location,Rating,Review Count,Price,Categories,Reviews,text
0,Bottega,"1132 Valencia St, San Francisco, CA 94158",4.3,1226,$$,"Italian, Pasta Shops, Pizza",1st time being here after wanting to go finall...,"Name: Bottega\nLocation: 1132 Valencia St, San..."
1,Fog Harbor Fish House,"39 Pier, San Francisco, CA 94133",4.4,11052,$$,"Seafood, Wine Bars, Cocktail Bars",It is hard to compare these foods but Fog Harb...,Name: Fog Harbor Fish House\nLocation: 39 Pier...
4,Marufuku Ramen,"1581 Webster St, Ste 235, San Francisco, CA 94115",4.5,5320,$$,Ramen,Delicious!\nThe broth was clear!\nThe seasonin...,Name: Marufuku Ramen\nLocation: 1581 Webster S...
5,Sotto Mare,"552 Green St, San Francisco, CA 94133",4.3,5489,$$,"Seafood, Italian, Bars",Really good ! It is worth it waiting a line !\...,"Name: Sotto Mare\nLocation: 552 Green St, San ..."
6,Blind Butcher,"4058 18th St, San Francisco, CA 94114",4.0,413,$$,"New American, Wine Bars","If you go to a restaurant called ""blind butche...","Name: Blind Butcher\nLocation: 4058 18th St, S..."
7,The Hungry Spot,"2077 Hayes St, San Francisco, CA 94117",4.9,45,$$,"Comfort Food, Salad, Sandwiches",This review effectively highlights key aspects...,Name: The Hungry Spot\nLocation: 2077 Hayes St...
9,Horsefeather,"528 Divisadero St, San Francisco, CA 94117",4.0,634,$$,"Cocktail Bars, New American",One of my favorite brunch spots in SF! I've vi...,Name: Horsefeather\nLocation: 528 Divisadero S...
12,Beit Rima,"86 Carl St, San Francisco, CA 94117",4.4,377,$$,"Arabic, Mediterranean, Lebanese",This place is amazing! I went with my boyfrien...,"Name: Beit Rima\nLocation: 86 Carl St, San Fra..."
13,Dalida,"101 Montgomery St, Ste 100, San Francisco, CA ...",4.5,329,$$$,"Mediterranean, Turkish, Bars",Very few places have it all - perfect location...,"Name: Dalida\nLocation: 101 Montgomery St, Ste..."
15,The Snug,"2301 Fillmore St, San Francisco, CA 94115",4.0,459,$$,"Cocktail Bars, New American",The Snug in Pacific Heights is a neighborhood ...,"Name: The Snug\nLocation: 2301 Fillmore St, Sa..."


In [29]:
df[df['Name']=='Fable']
df[df['Review Count']==7711]

,Name,Location,Rating,Review Count,Price,Categories,Reviews,text
32,Burma Superstar,"309 Clement St, San Francisco, CA 94118",4.2,7711,$$,Burmese,WHATTTTT! HOW HAVE I NOT WRITTEN A REVIEW FOR ...,Name: Burma Superstar\nLocation: 309 Clement S...


In [31]:
# Create documents from the DataFrame
documents = [Document(text=row['text']) for _, row in df.iterrows()]

# print(documents)
# Build the index
index = VectorStoreIndex.from_documents(documents)

# Query the index and generate a response
query = "Which restaurant is rated the highest? What is its rating? How many ratings does it have?"
eng = index.as_chat_engine()
response = eng.query(query)

# Output the response
print(response)

The highest-rated restaurant is "The Rustic" with a rating of 4.7. It has a total of 123 ratings.
